# Урок 3. CNN Свертки

## Домашнее задание

1. Обучите CNN (самописная) на CIFAR-100.

2. Обучите CNN на CIFAR-100 через дообучение ImageNet Resnet-50.

3. *Обучите CNN на CIFAR-100 через дообучение ImageNet Resnet-50 с аугментацией данных.

### Подключение библиотек

In [18]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets
from torch import nn
from torch.nn import functional as F
from torch.optim import Adam
from torchvision import models


from sklearn.model_selection import train_test_split

### Подготовка данных. Загрузка датасета CIFAR-100

In [3]:
dataset = datasets.CIFAR100(root='./data/', train=True, download=True)

Failed download. Trying https -> http instead. Downloading http://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz to ./data/cifar-100-python.tar.gz


  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting ./data/cifar-100-python.tar.gz to ./data/


In [4]:
class MyOwnCifar(Dataset):
   
    def __init__(self, init_dataset, transform=None):
        self._base_dataset = init_dataset
        self.transform = transform

    def __len__(self):
        return len(self._base_dataset)

    def __getitem__(self, idx):
        img = self._base_dataset[idx][0]
        if self.transform is not None:
            img = self.transform(img)
        label = self._base_dataset[idx][1]
        return img, label

In [5]:
def train_valid_split(dataset):
    X_train, X_test = train_test_split(dataset, test_size=0.05, random_state=13)
    return X_train, X_test

In [6]:
train_dataset, test_dataset = train_valid_split(dataset)

In [7]:
train_dataset = MyOwnCifar(train_dataset, transforms.ToTensor())

In [8]:
test_dataset = MyOwnCifar(test_dataset, transforms.ToTensor())

In [9]:
train_loader = DataLoader(train_dataset,
                          batch_size=100,
                          shuffle=True, 
                          num_workers=0)

test_loader = DataLoader(test_dataset,
                          batch_size=100,
                          shuffle=False,
                          num_workers=0)

In [10]:
classes = dataset.classes
len(classes)

100

In [11]:
for image, label in train_dataset:
    print(f'image size: {image.shape}\nimage_tensor:\n{image}\nclass: {classes[label]}')
    break

image size: torch.Size([3, 32, 32])
image_tensor:
tensor([[[0.6039, 0.6118, 0.6275,  ..., 0.4824, 0.4863, 0.4824],
         [0.6353, 0.6431, 0.6510,  ..., 0.5059, 0.5176, 0.5059],
         [0.6510, 0.6549, 0.6667,  ..., 0.5216, 0.5333, 0.5216],
         ...,
         [0.4314, 0.4588, 0.4980,  ..., 0.4588, 0.4902, 0.4510],
         [0.4392, 0.4627, 0.4902,  ..., 0.4784, 0.4784, 0.4510],
         [0.4392, 0.4588, 0.4745,  ..., 0.4745, 0.4588, 0.4314]],

        [[0.6000, 0.6078, 0.6235,  ..., 0.4627, 0.4627, 0.4627],
         [0.6314, 0.6392, 0.6471,  ..., 0.4863, 0.4902, 0.4902],
         [0.6471, 0.6510, 0.6627,  ..., 0.5020, 0.5059, 0.5020],
         ...,
         [0.3843, 0.4157, 0.4510,  ..., 0.4118, 0.4431, 0.4039],
         [0.3882, 0.4157, 0.4431,  ..., 0.4314, 0.4314, 0.4039],
         [0.3922, 0.4118, 0.4275,  ..., 0.4275, 0.4118, 0.3843]],

        [[0.6314, 0.6392, 0.6510,  ..., 0.4863, 0.4824, 0.4824],
         [0.6627, 0.6706, 0.6784,  ..., 0.5137, 0.5098, 0.5059],
        

### Обучение самописной CNN  на CIFAR-100.

In [14]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.bn1 = nn.BatchNorm2d(3) # 3 input channels
        self.conv1 = nn.Conv2d(3, 30, 3) # convolute 3 channels to 25 channels by frame of 3*3
                
        self.bn2 = nn.BatchNorm2d(30) # batchnorm of 25 channels
        self.conv2 = nn.Conv2d(30, 60, 3) # convolute 25 channels to 75 channels by frame of 3*3
        
        self.bn3 = nn.BatchNorm2d(60) # batchnorm of 75 channels
        self.conv3 = nn.Conv2d(60, 100, 2) # convolute 75 channels to 150 channels by frame of 2*2
        
        self.bn4 = nn.BatchNorm2d(100) # batchnorm of 150 channels
        self.conv4 = nn.Conv2d(100, 150, 2) # convolute 150 channels to 300 channels by frame of 2*2
        
        self.bn5 = nn.BatchNorm2d(150) # batchnorm of 300 channels
                
        self.dp1 = nn.Dropout(0.25) # dropout for 1 linear with 40% non-trainable params
        self.dp2 = nn.Dropout(0.2) # dropout for 2 linear with 30% non-trainable params
        self.dp3 = nn.Dropout(0.2) # dropout for 3 linear with 20% non-trainable params
        
        self.fc1 = nn.Linear(400, 250) # 1th linear layer, 1600 inputs -> 800 outputs
        self.fc2 = nn.Linear(450, 200) # 2th linear layer, 800 inputs -> 400 outputs
        self.fc3 = nn.Linear(400, 200) # 3th linear layer, 400 inputs -> 200 outputs
        self.out = nn.Linear(250, 100) # 4th linear layer, 200 inputs -> 100 outputs
            
    def forward(self, x):
        # 1 layer conv
        x = self.bn1(x)
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        
        # 2 layer conv
        x = self.bn2(x)
        x = self.conv2(x)
        x = F.leaky_relu(x)
        x = F.max_pool2d(x, 2)
        
        # 3 layer conv
        x = self.bn3(x)
        x = self.conv3(x)
        x = F.relu(x, 0.1)
        x = F.max_pool2d(x, 2)
        
        # 4 layer conv
        x = self.bn4(x)
                        
        # flatten
        x = x.view(x.size(0), -1) # transform x to vector
        
        # 5 layer linear
        x = self.dp1(x) 
        x = self.fc1(x)
        x = F.relu(x)
        
               
        return self.out(x)
       
net = Net()
net.cuda()
print(net)

Net(
  (bn1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): Conv2d(3, 30, kernel_size=(3, 3), stride=(1, 1))
  (bn2): BatchNorm2d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(30, 60, kernel_size=(3, 3), stride=(1, 1))
  (bn3): BatchNorm2d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(60, 100, kernel_size=(2, 2), stride=(1, 1))
  (bn4): BatchNorm2d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(100, 150, kernel_size=(2, 2), stride=(1, 1))
  (bn5): BatchNorm2d(150, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dp1): Dropout(p=0.25, inplace=False)
  (dp2): Dropout(p=0.2, inplace=False)
  (dp3): Dropout(p=0.2, inplace=False)
  (fc1): Linear(in_features=400, out_features=250, bias=True)
  (fc2): Linear(in_features=450, out_features=200, bias=True)
  (fc3): Linear(in_features=400, out_features=200, bias=True)

In [15]:
optimizer = Adam(net.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [17]:
num_epochs = 10
net.train()

for epoch in range(num_epochs):  
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].cuda(), data[1].cuda()

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        running_items += len(labels)
        running_right += (labels == torch.max(outputs, 1)[1]).sum()
        
        # выводим статистику о процессе обучения
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            net.eval()
            
            print(f'Epoch [{epoch + 1}/{num_epochs}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {running_loss / running_items:.3f}. ' \
                  f'Acc: {running_right / running_items:.3f}', end='. ')
            running_loss, running_items, running_right = 0.0, 0.0, 0.0

            test_running_right, test_running_total = 0.0, 0.0
            for i, data in enumerate(test_loader):
            
                test_outputs = net(data[0].cuda())
                test_running_total += len(data[1])
                test_running_right += (data[1].cuda() == torch.max(test_outputs, 1)[1]).sum()
            
            print(f'Test acc: {test_running_right / test_running_total:.3f}')
        
        net.train()
        
print('Training is finished!')

Epoch [1/10]. Step [1/475]. Loss: 0.048. Acc: 0.030. Test acc: 0.010
Epoch [1/10]. Step [301/475]. Loss: 0.038. Acc: 0.124. Test acc: 0.184
Epoch [2/10]. Step [1/475]. Loss: 0.033. Acc: 0.200. Test acc: 0.233
Epoch [2/10]. Step [301/475]. Loss: 0.031. Acc: 0.237. Test acc: 0.259
Epoch [3/10]. Step [1/475]. Loss: 0.028. Acc: 0.300. Test acc: 0.282
Epoch [3/10]. Step [301/475]. Loss: 0.028. Acc: 0.290. Test acc: 0.304
Epoch [4/10]. Step [1/475]. Loss: 0.027. Acc: 0.360. Test acc: 0.308
Epoch [4/10]. Step [301/475]. Loss: 0.026. Acc: 0.324. Test acc: 0.299
Epoch [5/10]. Step [1/475]. Loss: 0.024. Acc: 0.360. Test acc: 0.307
Epoch [5/10]. Step [301/475]. Loss: 0.025. Acc: 0.346. Test acc: 0.324
Epoch [6/10]. Step [1/475]. Loss: 0.025. Acc: 0.320. Test acc: 0.344
Epoch [6/10]. Step [301/475]. Loss: 0.024. Acc: 0.368. Test acc: 0.334
Epoch [7/10]. Step [1/475]. Loss: 0.024. Acc: 0.380. Test acc: 0.351
Epoch [7/10]. Step [301/475]. Loss: 0.024. Acc: 0.389. Test acc: 0.341
Epoch [8/10]. Step [

Самописная CNN показала метрику 0.35 на тесте

### Обучение CNN на CIFAR-100 через дообучение ImageNet Resnet-50

In [19]:
model = models.resnet50(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [22]:
for param in list(model.parameters())[:]:
    param.requires_grad = False

In [23]:
model.dp1 = nn.Dropout(0.2)
model.fc = nn.Linear(2048, 1024)
model.dp2 = nn.Dropout(0.2)
model.fc2 = nn.Linear(1024, 100)

In [24]:
trans_actions = transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]
                                  )
test_transforms = transforms.Compose([transforms.ToTensor(),
                                       transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]
                                     )

train_dataset, test_dataset = train_valid_split(dataset)

train_dataset = MyOwnCifar(train_dataset, trans_actions)
test_dataset = MyOwnCifar(test_dataset, test_transforms)

train_loader = DataLoader(train_dataset,
                          batch_size=100,
                          shuffle=True,
                          num_workers=0)
test_loader = DataLoader(test_dataset,
                         batch_size=100,
                         shuffle=False,
                         num_workers=0)

In [25]:
optimizer = Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [27]:
model.cuda()
num_epochs = 10
model.train()

for epoch in range(num_epochs):  
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].cuda(), data[1].cuda()

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        running_items += len(labels)
        running_right += (labels == torch.max(outputs, 1)[1]).sum()
        
        # выводим статистику о процессе обучения
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            model.eval()
            
            print(f'Epoch [{epoch + 1}/{num_epochs}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {running_loss / running_items:.3f}. ' \
                  f'Acc: {running_right / running_items:.3f}', end='. ')
            running_loss, running_items, running_right = 0.0, 0.0, 0.0

            test_running_right, test_running_total = 0.0, 0.0
            for i, data in enumerate(test_loader):
            
                test_outputs = model(data[0].cuda())
                test_running_total += len(data[1])
                test_running_right += (data[1].cuda() == torch.max(test_outputs, 1)[1]).sum()
            
            print(f'Test acc: {test_running_right / test_running_total:.3f}')
        
        model.train()
        
print('Training is finished!')

Epoch [1/10]. Step [1/475]. Loss: 0.134. Acc: 0.000. Test acc: 0.024
Epoch [1/10]. Step [301/475]. Loss: 0.071. Acc: 0.153. Test acc: 0.192
Epoch [2/10]. Step [1/475]. Loss: 0.079. Acc: 0.190. Test acc: 0.204
Epoch [2/10]. Step [301/475]. Loss: 0.069. Acc: 0.222. Test acc: 0.199
Epoch [3/10]. Step [1/475]. Loss: 0.061. Acc: 0.300. Test acc: 0.198
Epoch [3/10]. Step [301/475]. Loss: 0.069. Acc: 0.247. Test acc: 0.226
Epoch [4/10]. Step [1/475]. Loss: 0.055. Acc: 0.320. Test acc: 0.228
Epoch [4/10]. Step [301/475]. Loss: 0.068. Acc: 0.271. Test acc: 0.214
Epoch [5/10]. Step [1/475]. Loss: 0.056. Acc: 0.340. Test acc: 0.224
Epoch [5/10]. Step [301/475]. Loss: 0.067. Acc: 0.279. Test acc: 0.231
Epoch [6/10]. Step [1/475]. Loss: 0.057. Acc: 0.360. Test acc: 0.232
Epoch [6/10]. Step [301/475]. Loss: 0.067. Acc: 0.297. Test acc: 0.230
Epoch [7/10]. Step [1/475]. Loss: 0.058. Acc: 0.420. Test acc: 0.230
Epoch [7/10]. Step [301/475]. Loss: 0.066. Acc: 0.307. Test acc: 0.229
Epoch [8/10]. Step [

ImageNet Resnet-50 показывает метрику на тесте ниже (0.24), чем самописная